In [64]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from seleniumbase import Driver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium_stealth import stealth
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import re # Expresiones regulares 
import time
import pandas as pd
import random as rd

In [65]:
url = 'https://musicstax.com/'
service = Service(executable_path='scrapping\chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

In [66]:
# Este apartado es para acceder al enlace y saltarnos el captcha que comprueba que somos humanos
driver = Driver(uc=True, headless=False)

url = "https://musicstax.com/"

driver.uc_open_with_reconnect(url, reconnect_time=6)


In [ ]:
# Cargamos el listado de playlists
playlists = pd.read_csv('scrapping\playlist.csv')

In [68]:
# Realiza la búsqueda de las playlists en el searchbox de la página
def search_playlist(i):
    action = ActionChains(driver)
    search_box = driver.find_element(By.CSS_SELECTOR, 'body > div.container > header > div.menu-right > div.menu-container.menu-search.menu-search-bar > form > div > div > input') #searchbox
    time.sleep(1.3)
    action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()
    time.sleep(0.5)
    search_click = driver.find_element(By.CSS_SELECTOR, 'body > div.container > header > div.menu-right > div.menu-container.menu-search.menu-search-bar > form > div > button')
    time.sleep(0.5)
    search_click.click()

In [69]:
def scroll(n):
    current_height = driver.execute_script("return window.pageYOffset")

    while current_height < n:
        time.sleep(rd.uniform(0.5,1.2))
        driver.execute_script("window.scrollBy(0, 100)")
        current_height = driver.execute_script("return window.pageYOffset")
    return

In [71]:
def move_to_song():
    actions = ActionChains(driver)

    # Obtener las coordenadas del elemento
    element_location = element.location
    current_location = driver.execute_script("return {x: window.scrollX, y: window.scrollY};")

    # Calculamos la cantidad de pasos
    steps = 10
    x_step = (element_location['x'] - current_location['x']) / steps
    y_step = (element_location['y'] - current_location['y']) / steps

    # El desplazamiento se hace en pasos pequeños para humanizar el movimiento lo máximo posible
    for i in range(steps):
        driver.execute_script(f"window.scrollBy({x_step}, {y_step});")
        time.sleep(rd.uniform(0.1,0.5))  # Pequeña pausa entre cada desplazamiento
    driver.execute_script("window.scrollBy(0, -200);")

In [72]:
def click_song():
    click_s= driver.find_element(By.CSS_SELECTOR, css_selector)
    time.sleep(0.5)
    click_s.click()
    time.sleep(2.1)
    url_actual = driver.current_url
    # Comprueba si es la página de error 404
    if url_actual == "https://musicstax.com/404":
        time.sleep(0.5)  # Descanso breve antes de continuar
    else: 
        scroll(290)


In [70]:
# Extraemos todo el metadato de las canciones de la playlist
def extract_info():
    html_source = driver.page_source
    soup = bs(html_source, 'html.parser')
    
    song_id = (soup.find(class_="song-discover-similar").a['href']).split('/')[-1]
    song_dict.update({'id': song_id})

    info_tech = soup.find_all(class_="song-fact-container-stat")

    keys_tech = ['tempo', 'key', 'loudness', 'beat']
    values_tech = [item.get_text().replace('\n', '').replace('\t', '').strip() for item in info_tech]

    tech_dict = {keys_tech[i]: values_tech[i] for i in range(len(keys_tech))}
    song_dict.update(tech_dict)

    info_music = (soup.find_all(class_="song-bar-stat"))

    keys_music = ['popularity', 'energy', 'danceability', 'positiveness','speechiness', 'liveness', 'acousticness','instrumentalness']
    values_music = [item.get_text().replace('\n', '').replace('\t', '').strip() for item in info_music]

    music_dict = {keys_music[i]: values_music[i] for i in range(len(keys_music))}

    song_dict.update(music_dict)


In [73]:
#obtener el último índice para automatizar la extracción
def top_index():
    html_source_listado = driver.page_source
    soup_list = bs(html_source_listado, 'html.parser')
    last_song = (soup_list.find_all(class_="album-track-number"))
    last_index = int(last_song[-1].get_text().replace('\n', '').replace('\t', '').strip())*2
    return last_index

In [ ]:
songs_details = []
for i in range(0, 71, 1): # i devuelve la playlist para la función search_playlist
    search_playlist(i)
    last_index = top_index()
    for i in range(2, last_index+2,2): # el id de cada canción sobre el que iteramos para navegar hasta ella y clicarla es siempre par
        song_dict={}
        css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(i) + ') > div.album-track-number')
        element = driver.find_element(By.CSS_SELECTOR, css_selector)
        move_to_song()
        time.sleep(rd.uniform(0.5,1.2))
        click_song()
        time.sleep(rd.uniform(0.5,1.2))
        try:
            extract_info()
            print(f"Info Canción nº "+str(int(i/2))+" extraída")
        except AttributeError as e:
            print(f"Info Canción nº "+str(int(i/2))+" no disponible")
            time.sleep(0.5)
            #driver.back()
        time.sleep(rd.uniform(0.5,1.2))
        songs_details.append(song_dict)
        driver.back()
    #pasar a escalares todos las keys
df_details = pd.DataFrame(songs_details)
df_details.to_csv('songs_details.csv')
driver.quit()